In [12]:
#STEP 0 -  Load env 
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
# STEP 1 — Environment Setup
import os
from langchain_openai import ChatOpenAI

# Deterministic LLM (enterprise rule)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=300
)

Why temperature = 0?

Enterprise requires reproducibility.

In [14]:
# STEP 2 — Controlled Database Access
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=["Track", "Album", "Artist"]
)

Why include_tables?

Never expose full DB in production.

Scope control is mandatory.

In [20]:
db

In [22]:
llm 

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'text_inputs': True, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000000000C441930>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000000000C440F10>, root_client=<openai.OpenAI object at 0x000000000C441DE0>, root_async_client=<openai.AsyncOpenAI object at 0x000000000C441E70>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, max_tokens=300)

In [15]:
# STEP 3 — SQLToolkit (Explicit Tool Control)
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(
    db=db,
    llm=llm
)

tools = toolkit.get_tools()

This gives you:

sql_db_query

sql_db_schema

sql_db_list_tables

sql_db_query_checker

But now YOU control them.

In [23]:
tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000000000C441AB0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000000000C441AB0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000000000C441AB0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [16]:
# STEP 4 — Add Safety Layer
def validate_query(query: str):
    forbidden_keywords = ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER"]
    for keyword in forbidden_keywords:
        if keyword in query.upper():
            raise ValueError(f"Unsafe query detected: {keyword} not allowed.")
    return True

Enterprise SQL agent must never blindly execute queries.

In [17]:
# STEP 5 — Structured Tool Calling (Hybrid Design)
from langchain_core.messages import ToolMessage

# Bind toolkit tools
llm_with_tools = llm.bind_tools(tools)

In [24]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'text_inputs': True, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000000000C441930>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000000000C440F10>, root_client=<openai.OpenAI object at 0x000000000C441DE0>, root_async_client=<openai.AsyncOpenAI object at 0x000000000C441E70>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, max_tokens=300), kwargs={'tools': [{'type': 'function', 'function': {'name': 'sql_db_query'

In [28]:
# STEP 6 — Controlled Execution Loop
from langchain_core.messages import ToolMessage

def run_enterprise_sql_agent(question: str):

    messages = [
        ("system", 
         "You are an enterprise SQL analyst. "
         "Use only SELECT queries. "
         "Use exact table names."),
        ("human", question)
    ]

    while True:
        response = llm_with_tools.invoke(messages)

        # If model wants to call a tool
        if response.tool_calls:
            tool_call = response.tool_calls[0]
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # If query tool
            if "query" in tool_args:
                validate_query(tool_args["query"])

            # Execute tool
            tool = next(t for t in tools if t.name == tool_name)
            result = tool.invoke(tool_args)

            # Append tool result to conversation
            messages.append(response)
            messages.append(
                ToolMessage(
                    content=str(result),
                    tool_call_id=tool_call["id"]
                )
            )

            continue  # Loop again

        # If no tool call → final answer
        return response.content

Step 1 → Wrong SQL

Step 2 → SQL error returned

Step 3 → Model decides to list tables

Step 4 → Tables returned

Step 5 → Model generates correct SQL

Step 6 → SQL executed

Step 7 → Final answer returned

In [29]:
# STEP 7 — Test
answer = run_enterprise_sql_agent(
    "How many tracks are there in the database?"
)

print(answer)

There are 3503 tracks in the database.
